In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Let's load the data for the first time
df = pd.read_pickle(os.path.join('data', 'artwork_data.pickle'))

In [3]:
# ITERATION
small_df = df.iloc[49980:50019, :].copy()
small_df

,artist,title,medium,year,acquisitionYear,width,height,units
id,,,,,,,,
16412,Wols,[no title],Etching on paper,1937.0,1983.0,130,80,mm
16413,Wols,[no title],Etching and drypoint on paper,1937.0,1983.0,187,102,mm
16414,Wols,[no title],Etching and drypoint on paper,1937.0,1983.0,248,168,mm
16415,Wols,[no title],Etching and drypoint on paper,1937.0,1983.0,149,102,mm
16416,Wols,[no title],Etching and drypoint on paper,1937.0,1983.0,203,120,mm
16417,Wols,[no title],Etching on paper,1937.0,1983.0,130,79,mm
16418,Wols,[no title],Etching and drypoint on paper,1937.0,1983.0,124,98,mm
16419,Wols,[no title],Etching and drypoint on paper,1937.0,1983.0,140,89,mm
16420,Wols,[no title],Etching and drypoint on paper,1937.0,1983.0,140,86,mm


In [4]:
grouped = small_df.groupby('artist')
grouped

In [5]:
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

In [6]:
for name, group_df in grouped:
    print(name)
    print('----------------------')
    print(group_df)
    break

Frost, Sir Terry
----------------------
                artist            title               medium    year  \
id                                                                     
4704  Frost, Sir Terry        Blue Moon  Lithograph on paper  1952.0   
4705  Frost, Sir Terry      Boat Shapes     Linocut on paper  1952.0   
4706  Frost, Sir Terry      Boat Shapes     Linocut on paper  1954.0   
4707  Frost, Sir Terry      Boat Shapes     Linocut on paper  1954.0   
4708  Frost, Sir Terry            Leeds    Drypoint on paper  1956.0   
4709  Frost, Sir Terry  Camping, Anduze     Etching on paper  1979.0   
4710  Frost, Sir Terry     Umea, Sweden     Etching on paper  1979.0   
4711  Frost, Sir Terry    Self-Portrait     Etching on paper  1980.0   

      acquisitionYear width height units  
id                                        
4704           1983.0   355    273    mm  
4705           1983.0   132    143    mm  
4706           1983.0   131    155    mm  
4707           1983.0   

In [7]:
# Aggregate
# Mins
for name, group_df in small_df.groupby('artist'): 
    min_year = group_df['acquisitionYear'].min()
    print("{}: {}".format(name, min_year))

Frost, Sir Terry: 1983.0
Phillips, Esq Tom: 1983.0
Wols: 1983.0


In [8]:
# Transform
# Equivalent of editing by hand:
# Make a case when there is no data to infer
# small_df.loc[[11838, 16441], 'medium'] = np.nan
def fill_values(series):
    values_counted = series.value_counts()
    if values_counted.empty:
        return series
    most_frequent = values_counted.index[0]
    new_medium = series.fillna(most_frequent)
    return new_medium

In [9]:
def transform_df(source_df):
    group_dfs = []  
    for name, group_df in source_df.groupby('artist'):
        filled_df = group_df.copy()
        filled_df.loc[:, 'medium'] = fill_values(group_df['medium'])
        group_dfs.append(filled_df)
    
    new_df = pd.concat(group_dfs)
    return new_df

In [10]:
# Now check the result
filled_df = transform_df(small_df)
filled_df

,artist,title,medium,year,acquisitionYear,width,height,units
id,,,,,,,,
4704,"Frost, Sir Terry",Blue Moon,Lithograph on paper,1952.0,1983.0,355,273,mm
4705,"Frost, Sir Terry",Boat Shapes,Linocut on paper,1952.0,1983.0,132,143,mm
4706,"Frost, Sir Terry",Boat Shapes,Linocut on paper,1954.0,1983.0,131,155,mm
4707,"Frost, Sir Terry",Boat Shapes,Linocut on paper,1954.0,1983.0,193,267,mm
4708,"Frost, Sir Terry",Leeds,Drypoint on paper,1956.0,1983.0,125,167,mm
4709,"Frost, Sir Terry","Camping, Anduze",Etching on paper,1979.0,1983.0,257,209,mm
4710,"Frost, Sir Terry","Umea, Sweden",Etching on paper,1979.0,1983.0,NaN,NaN,NaN
4711,"Frost, Sir Terry",Self-Portrait,Etching on paper,1980.0,1983.0,277,200,mm
11838,"Phillips, Esq Tom",[colophon],Etching on paper,1979.0,1983.0,292,204,mm


In [11]:
# BUILT-INS
# Transform
grouped_mediums = small_df.groupby('artist')['medium']
grouped_mediums

In [12]:
small_df.loc[:, 'medium'] = grouped_mediums.transform(fill_values)

In [16]:
# Min
df.groupby('artist')['acquisitionYear'].agg(np.min)

artist
?British School           1927.0
Abakanowicz, Magdalena    2009.0
Abbey, Edwin Austin       1924.0
Abbott, Berenice          2010.0
Abbott, Lemuel Francis    1885.0
                           ...  
Zuloaga, Ignacio          1923.0
Zyw, Aleksander           1962.0
di Suvero, Mark           2004.0
van Elk, Ger              1980.0
Štyrský, Jindrich         2007.0
Name: acquisitionYear, Length: 3336, dtype: float64

In [17]:
df.groupby('artist')['acquisitionYear'].min()

artist
?British School           1927.0
Abakanowicz, Magdalena    2009.0
Abbey, Edwin Austin       1924.0
Abbott, Berenice          2010.0
Abbott, Lemuel Francis    1885.0
                           ...  
Zuloaga, Ignacio          1923.0
Zyw, Aleksander           1962.0
di Suvero, Mark           2004.0
van Elk, Ger              1980.0
Štyrský, Jindrich         2007.0
Name: acquisitionYear, Length: 3336, dtype: float64

In [19]:
# Filter
grouped_titles = df.groupby('title')
title_counts = grouped_titles.size().sort_values(ascending=False)
title_counts

title
[title not known]                                                          3572
Blank                                                                      3031
[blank]                                                                    2482
[no title]                                                                 1874
Untitled                                                                    639
                                                                           ... 
Gondorf [Turner]                                                              1
Good                                                                          1
Good Boy Bad Boy                                                              1
Good Enough to Eat                                                            1
“What is to be done?” 1984. Alternative Technology Versus Nuclear Power       1
Length: 43531, dtype: int64

In [20]:
dup_titles_df = grouped_titles.filter(lambda x: len(x.index) > 1)
dup_titles_df.sort_values('title', inplace=True)
dup_titles_df

,artist,title,medium,year,acquisitionYear,width,height,units
id,,,,,,,,
20571,"Scully, Sean",#1,Aquatint on paper,1992.0,1992.0,117,174,mm
21345,"Dunham, Carroll",#1,Drypoint on paper,1989.0,2002.0,392,579,mm
21346,"Dunham, Carroll",#2,Drypoint on paper,1989.0,2002.0,392,579,mm
20572,"Scully, Sean",#2,Aquatint on paper,1992.0,1992.0,323,278,mm
21347,"Dunham, Carroll",#3,Drypoint on paper,1989.0,2002.0,392,579,mm
...,...,...,...,...,...,...,...,...
97431,"Warhol, Andy",“The Runaway Pigeon”,Ink on paper on ink on paper,1953.0,2009.0,280,205.0,mm
97408,"Warhol, Andy",“The Runaway Pigeon”,"Ink, watercolour and graphite on paper",1953.0,2009.0,269,295.0,mm
97409,"Warhol, Andy",“The Runaway Pigeon”,"Ink, watercolour and graphite on paper",1953.0,2009.0,272,237.0,mm
